In [64]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import numpy as np
import torch
from typing import List
from tqdm import tqdm

In [3]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [4]:
expla_df = pd.read_csv("../data/explagraphs/train_v3.tsv", sep="\t")
copa_df = pd.read_csv("../data/copa/train_v3.tsv", sep="\t")

In [7]:
columns = {
    'linked_paths': 'linked_paths',
    'gold_graph': 'gold_graph',
    'generated_graph_linked': 'generated_graph_linked',
    'generated_graph_gold': 'generated_graph_gold',
    'retrieved_graph': 'retrieved_graph'
}

In [31]:
def string_stripper(s: str):
    return s.strip().replace('[', '').replace(']', '').replace(',', '').replace('_', '').replace('\'', '')

In [71]:
def get_avg_similarity(df, is_expla=False):
    stats = {}
    for key, value in columns.items():
        l = df[value].to_numpy().tolist()
        res = []
        for idx, e in enumerate(tqdm(l)):
            try:
                e = string_stripper(e)
                if is_expla:
                    context = df.iloc[idx]["belief"] + " " + df.iloc[idx]["argument"]
                else:
                    context= df.iloc[idx]["p"] + " " + df.iloc[idx]["a1"] + " " + df.iloc[idx]["a2"]
            except:
                continue
            c = model.encode(context, convert_to_tensor=True, show_progress_bar=False)
            g = model.encode(e, convert_to_tensor=True, show_progress_bar=False)
            res.append(util.cos_sim(c,g).item())            
            
        stats[value] = np.mean(res)            
        
    return stats
        

In [66]:
a = get_avg_similarity(expla_df, is_expla=True)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2238/2238 [00:36<00:00, 61.33it/s]


In [67]:
a

{'linked_paths': 0.45119431000735555,
 'gold_graph': 0.7517270463534612,
 'generated_graph_linked': 0.48015500254570354,
 'generated_graph_gold': 0.5591106350854925,
 'retrieved_graph': 0.39514586703112753}

In [72]:
b = get_avg_similarity(copa_df, is_expla=False)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1215/1215 [00:21<00:00, 57.24it/s]


In [73]:
b

{'linked_paths': 0.42919589884027287,
 'gold_graph': 0.5715929444189425,
 'generated_graph_linked': 0.45180113922819926,
 'generated_graph_gold': 0.46292619898293896,
 'retrieved_graph': 0.3266092926401783}